In [1]:
import pandas as pd
import google.generativeai as genai
import time
from tqdm import tqdm
import json
import time
import os

C:\Users\luizm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\luizm\AppData\Local\Temp\ipykernel_15280\262575556.py:2: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  import google.generativeai as genai


In [ ]:
# --- CONFIGURAÇÃO ---
# Substitua pela sua chave REAL da Google AI
API_KEY = "substiruir por uma chave da API válida"
INTERVALO_SEGURANCA = 13  # Intervalo entre requisições para evitar rate limits

INPUT_PATH = '../data/olist_processed.parquet'
OUTPUT_PATH = '../data/olist_enriched_gemini.parquet'

In [3]:
# # Configurando o cliente (versão nova da lib)
# client = openai.OpenAI(api_key=OPENAI_API_KEY)
# Configuração para garantir que a saída seja JSON limpo
generation_config = {
    "temperature": 0,
    "response_mime_type": "application/json"
}

model = genai.GenerativeModel(
    model_name="gemini-2.5-flash",
    generation_config=generation_config,
)

In [4]:
genai.configure(api_key=API_KEY)

In [5]:
# --- 1. CARREGAR DADOS ---
print("Carregando dados...")
df = pd.read_parquet(INPUT_PATH)

# Filtrar apenas linhas que TEM review escrito (muitos são nulos)
# E pegar apenas colunas relevantes para economizar memória
df_reviews = df.dropna(subset=['review_comment_message']).copy()

# ⚠️ AMOSTRA: Pegando 50 reviews para demonstração (Economia de Custo/Tempo)
df_sample = df_reviews.head(14).copy()

print(f"Total de reviews com texto: {len(df_reviews)}")
print(f"Processando amostra de: {len(df_sample)} registros para PoC de IA.")

Carregando dados...
Total de reviews com texto: 47642
Processando amostra de: 14 registros para PoC de IA.


In [ ]:
def analisar_sentimento_mock(comentario):
    """
    Simula uma chamada de API de IA analisando palavras-chave.
    Isso garante que o pipeline funcione sem depender de créditos externos.
    """
    if not isinstance(comentario, str):
        return {"sentimento": "neutro", "motivo": "outros"}
        
    comentario = comentario.lower()
    
    # Lógica de "Sentimento" - CORRIGIDA AQUI (adicionado 'for x in')
    if any(x in comentario for x in ['bom', 'ótimo', 'otimo', 'excelente', 'adorei', 'gostei', 'recomendo', 'rápido', 'rapido']):
        sentimento = "positivo"
    elif any(x in comentario for x in ['ruim', 'péssimo', 'pessimo', 'horrível', 'horrivel', 'demora', 'atraso', 'não recebi', 'defeito']):
        sentimento = "negativo"
    else:
        sentimento = "neutro"
        
    # Lógica de "Motivo" - CORRIGIDA AQUI
    if any(x in comentario for x in ['entrega', 'correio', 'chegou', 'prazo', 'atrasou', 'recebi', 'veio']):
        motivo = "entrega"
    elif any(x in comentario for x in ['produto', 'qualidade', 'quebrado', 'funcionou', 'bonito', 'feio']):
        motivo = "produto"
    elif any(x in comentario for x in ['atendimento', 'vendedor', 'loja', 'resposta']):
        motivo = "atendimento"
    else:
        motivo = "outros"
        
    return {"sentimento": sentimento, "motivo": motivo}

In [6]:
# --- 2. FUNÇÃO DE IA (GEMINI) ---
def analisar_sentimento_gemini(comentario):
    prompt = f"""
    Você é um especialista em Customer Experience (CX).
    Analise o comentário abaixo de um cliente de e-commerce.
    
    Comentário: "{comentario}"
    
    Retorne um objeto JSON com as seguintes chaves:
    - "sentimento": "positivo", "negativo" ou "neutro"
    - "motivo": "entrega", "produto", "qualidade", "atendimento" ou "outros"
    """
    
    try:
        response = model.generate_content(prompt)
        # Como forçamos JSON no config, podemos carregar direto
        return json.loads(response.text)
    except Exception as e:
        print(f"Erro ao processar: {e}")
        # Retorna um fallback em caso de bloqueio de segurança ou erro
        return {"sentimento": "erro", "motivo": "erro"}

In [8]:
print(f"Iniciando processamento de {len(df_sample)} registros com limite de 5 RPM...")
tempo_estimado_min = (len(df_sample) * INTERVALO_SEGURANCA) / 60
print(f"⚠️ Estimativa de tempo total: {tempo_estimado_min:.1f} minutos. Tenha paciência!")

resultados = []

for index, row in tqdm(df_sample.iterrows(), total=df_sample.shape[0]):
    coment = row['review_comment_message']
    
    sucesso = False
    tentativas = 0
    
    while tentativas < 3 and not sucesso:
        try:
            # Chama a função definida no passo anterior
            res = analisar_sentimento_gemini(coment)
            resultados.append(res)
            sucesso = True
        except Exception as e:
            print(f"\nErro no índice {index}: {e}. Aguardando 20s para tentar de novo...")
            time.sleep(20) # Backoff maior em caso de erro
            tentativas += 1
            
    if not sucesso:
        resultados.append({"sentimento": "erro", "motivo": "erro_api"})

    # O FREIO DE MÃO PUXADO
    time.sleep(INTERVALO_SEGURANCA)

# Transforma em colunas
df_resultados = pd.DataFrame(resultados)
df_sample['sentimento_ia'] = [r.get('sentimento') for r in resultados]
df_sample['motivo_ia'] = [r.get('motivo') for r in resultados]

print("\nConcluído!")

Iniciando processamento de 14 registros com limite de 5 RPM...
⚠️ Estimativa de tempo total: 3.0 minutos. Tenha paciência!


  0%|          | 0/14 [00:00<?, ?it/s]

 93%|█████████▎| 13/14 [03:25<00:16, 16.52s/it]

Erro ao processar: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash
Please retry in 16.061000306s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 20
}
, retry_delay {
  seconds: 16
}
]


100%|██████████| 14/14 [03:39<00:00, 15.66s/it]


Concluído!


In [9]:
df_sample_corr = df_sample.copy()
df_sample_corr['sentimento_ia'] = [r.get('sentimento') for r in resultados]
df_sample_corr['motivo_ia'] = [r.get('motivo') for r in resultados]

In [10]:
# --- 4. VISUALIZAÇÃO DOS RESULTADOS ---
print("\n=== ANTES (Texto Puro) vs DEPOIS (Dados Estruturados) ===")
display(df_sample[['review_comment_message', 'sentimento_ia', 'motivo_ia']].head(10))


=== ANTES (Texto Puro) vs DEPOIS (Dados Estruturados) ===


,review_comment_message,sentimento_ia,motivo_ia
0,"Não testei o produto ainda, mas ele veio corre...",negativo,entrega
1,Muito bom o produto.,positivo,produto
3,O produto foi exatamente o que eu esperava e e...,positivo,entrega
6,fiquei triste por n ter me atendido.,negativo,atendimento
10,Aguardando retorno da loja,negativo,atendimento
11,Aguardando retorno da loja,negativo,atendimento
15,Gostei do produto,positivo,produto
16,Obrigado pela atenção. Lojas lannister perfeit...,positivo,atendimento
21,os correios estäo em greve... näo recebi nenhu...,negativo,atendimento
25,Sempre vou comprar aqui pois é a melhor parabéns,positivo,outros


In [11]:
# --- 5. MERGE E SALVAMENTO ---
# Como pegamos apenas uma amostra, vamos salvar separado para o Dashboard
# (No dashboard, vamos mostrar "Análise de Sentimento (Amostra em Tempo Real)")
df_sample.to_parquet(OUTPUT_PATH, index=False)
print(f"\nBase enriquecida salva em: {OUTPUT_PATH}")


Base enriquecida salva em: ../data/olist_enriched_gemini.parquet
